# 2.01 - Airline Routes Dataset

## **Source_Datasets**

### **airline_routes.tsv**
    As of June 2014, the OpenFlights/Airline Route Mapper Route Database contains 67663 routes between 3321 airports on 548 airlines spanning the globe, as shown in the map above. 
- Source: [link](https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat)
- MimeType: /text/csv
- Columns:
    - `Airline`	                2-letter (IATA) or 3-letter (ICAO) code of the airline.
    - `Airline_ID`	            Unique OpenFlights identifier for airline (see Airline).
    - `Source_airport`	        3-letter (IATA) or 4-letter (ICAO) code of the source airport.
    - `Source_airport_ID`	    OpenFlights identifier for source airport    
    - `Destination_airport`	    3-letter (IATA) or 4-letter (ICAO) code of the destination airport.
    - `Destination_airport_ID`	OpenFlights identifier for destination
    - `Codeshare`	            "Y" if this flight is a codeshare (not operated by Airline, but another carrier), empty otherwise.
    - `Stops`                   Number of stops on this flight ("0" for direct)
    - `Equipment`	            3-letter codes for plane type(s) generally used on this flight, separated by spaces

### **airlines_planes.tsv**
    The OpenFlights plane database contains a curated selection of 173 passenger aircraft with IATA and/or ICAO codes, covering the vast majority of flights operated today and commonly used in flight schedules and reservation systems. Each entry contains the following information:
- Source: [link](https://raw.githubusercontent.com/jpatokal/openflights/master/data/planes.dat)
- MimeType: /text/csv
- Columns:
    - `Name`	            Full name of the aircraft.
    - `IATA_Code`	        Unique three-letter IATA identifier for the aircraft.
    - `ICAO_Code`	        Unique four-letter ICAO identifier for the aircraft.

### **airlines_airpots.tsv**
- Source: [link](https://ourairports.com/data/)
- MimeType: /text/csv
- Columns:
    - `ID`	                description
    - `Ident`	            description
    - `Type`	            description
    - `Name`                description
    - `Latitude_Deg`        description
    - `Longitude_Deg`	    description
    - `Elevation_Ft`	    description
    - `Continent`	        description
    - `ISO_Country`         description
    - `ISO_Region`          description
    - `Municipality`	    description
    - `Scheduled_Service`	description
    - `ICAO_Code`	        description
    - `IATA_Code`           description
    - `GPS_Code`            description
    - `Local_Code`	        description
    - `Home_Link`	        description
    - `Wikepedia_Link`      description
    - `Keywords`            description


## Output Files
### **american_routes.tsv**
- `Plane Model`         airlines_planes.tsv
- `Longitude_Source`    airlines_airpots.tsv
- `Latitude_Source`
- `Longitude_Dest`
- `Latitude_Dest`
- `Country_Source`
- `Country_Dest`



In [ ]:
import pandas as pd
import numpy as np
import requests
from io import StringIO

### Airline_routes.tsv

In [ ]:
## URL ## 
routes_url = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat"

## Download ##
response = requests.get(routes_url)

columns = [
    "Airline",
    "Airline_ID",
    "Source_Airport",
    "Source_Airport_ID",
    "Destination_Airport",
    "Destination_Airport_ID",
    "Codeshare",
    "Stops",
    "Equipment"
]

## Save as TSV ##
df = pd.read_csv(StringIO(response.text), header=None, names = columns, index_col= False)

df.to_csv('../data/joined_datasets/airline_routes.tsv', sep = '\t', index = False)


### airline_planes.tsv

In [ ]:
# URL of the data
planes_url = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/planes.dat"

# Download the file
response = requests.get(planes_url)

columns = [
    "Name",
    "IATA_Code",
    "ICAO_Code",
]

df = pd.read_csv(StringIO(response.text), header=None, names = columns, index_col= False)
df.to_csv('../data/joined_datasets/airline_planes.tsv', sep = '\t', index = False)


### airlines_airports.tsv

In [ ]:
# Used csv2tab to convert to tsv
df = pd.read_csv('../data/joined_datasets/airline_airports.tsv', sep = "\t")
df.info()

### Combining 3 datasets

In [ ]:
df_airports = pd.read_csv('../data/joined_datasets/airline_airports.tsv', sep = "\t")
df_planes = pd.read_csv('../data/joined_datasets/airline_planes.tsv', sep = "\t")
df_routes = (pd.read_csv('../data/joined_datasets/airline_routes.tsv', sep = "\t")
                .astype({
                    "Airline": str,
                    "Airline_ID": str,
                    "Source_Airport": str,
                    "Source_Airport_ID": str,
                    "Destination_Airport": str,
                    "Destination_Airport_ID": str,
                    "Codeshare": str,
                    "Stops": str, 
                    "Equipment": str   
                })
)
df_routes.head()

In [ ]:
df_routes.columns

In [306]:
american_airports = (df_airports
                    .loc[df_airports["Iso_Country"] == "US"]
                    .drop(["Continent", "Wikipedia_Link", "Keywords", "Home_Link"], axis = 1)
                    .fillna({
                        "Iata_Code": 0,
                        "Icao_Code": 0,
                        "GPS_Code": 0,
                        "Local_Code": 0
                    })
                    .astype({
                        "Name": str,
                        "Iata_Code": str,
                        "Icao_Code": str,
                        "Gps_Code": str,
                        "Local_Code": str,
                        "Latitude_Deg": float,
                        "Longitude_Deg": float,
                        "Elevation_Ft": float,
                        
                    })
)


valid_american_airports = american_airports.loc[american_airports['Iata_Code'] != '0']

american_Iata_Codes = valid_american_airports['Iata_Code'].to_list()
american_Iata_Codes[:10]

american_routes = df_routes[df_routes['Source_Airport'].isin(american_Iata_Codes) | df_routes['Destination_Airport'].isin(american_Iata_Codes)]

In [293]:
## Iata_Codes for American Airports ##
american_Iata_Codes = american_airports.loc[american_airports['Iata_Code'] != '0', 'Iata_Code'].to_list()
american_Iata_Codes[:10]

## Select American Routes ##
american_routes = df_routes[df_routes['Source_Airport'].isin(american_Iata_Codes) & df_routes['Destination_Airport'].isin(american_Iata_Codes)]

## Initialize Outfile ##
df_out = american_routes.copy()

## Add Source Longitude, Latitude, Airport Type ##
df_out = df_out.merge(df_airports[["Latitude_Deg","Longitude_Deg","Type", "Iso_Country","Iata_Code"]],
                      left_on = "Source_Airport", 
                      right_on = "Iata_Code",
                      how = 'left',
                      ).rename({"Latitude_Deg": "Latitude_Source", "Longitude_Deg" : "Longitude_Source", "Type" : "Type_Source", "Iso_Country": "Country_Source"}, axis = 1)

## Add Destination Longitude, Latitude, Airport Type ##
df_out = df_out.merge(df_airports[["Latitude_Deg","Longitude_Deg","Type", "Iso_Country","Iata_Code"]],
                      left_on = "Destination_Airport", 
                      right_on = "Iata_Code",
                      how = 'left',
                      ).rename({"Latitude_Deg": "Latitude_Dest", "Longitude_Deg" : "Longitude_Dest", "Type" : "Type_Dest", "Iso_Country": "Country_Dest"}, axis = 1)

## Drop Redundant Columns ##
df_out.drop(["Iata_Code_x", "Iata_Code_y"], axis = 1, inplace = True)

df_out[df_out['Source_Airport'] == 'LAX']

,Airline,Airline_ID,Source_Airport,Source_Airport_ID,Destination_Airport,Destination_Airport_ID,Codeshare,Stops,Equipment,Latitude_Source,Longitude_Source,Type_Source,Country_Source,Latitude_Dest,Longitude_Dest,Type_Dest,Country_Dest
77,4B,20565,LAX,3484,LAS,3877,nan,0,PL2,33.942501,-118.407997,large_airport,US,36.083361,-115.151817,large_airport,US
1274,AA,24,LAX,3484,ABQ,4019,Y,0,CRJ CR7,33.942501,-118.407997,large_airport,US,35.039976,-106.608925,large_airport,US
1275,AA,24,LAX,3484,ANC,3774,Y,0,737,33.942501,-118.407997,large_airport,US,61.179004,-149.992561,large_airport,US
1276,AA,24,LAX,3484,AUS,3673,nan,0,M83 738,33.942501,-118.407997,large_airport,US,30.197535,-97.662015,large_airport,US
1277,AA,24,LAX,3484,BDL,3825,nan,0,738,33.942501,-118.407997,large_airport,US,41.938510,-72.688066,large_airport,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10046,WN,4547,LAX,3484,TUS,3636,nan,0,73W 73C 733 73H,33.942501,-118.407997,large_airport,US,32.115004,-110.938053,medium_airport,US
10675,ZK,2607,LAX,3484,IGM,6132,nan,0,BE1,33.942501,-118.407997,large_airport,US,35.259499,-113.938004,medium_airport,US
10676,ZK,2607,LAX,3484,MCE,7122,nan,0,BE1,33.942501,-118.407997,large_airport,US,37.284698,-120.514000,medium_airport,US
10677,ZK,2607,LAX,3484,PRC,3446,nan,0,BE1,33.942501,-118.407997,large_airport,US,34.654499,-112.419998,medium_airport,US


### Trajectory Calculation

In [ ]:

import math

def cartesian_convert(lat,lon):
    '''
    Converts latitude and longitude to cartesian coordinates.
    '''
    lat = math.radians(lat)
    lon = math.radians(lon)
    x = math.cos(lat) * math.cos(lon)
    y = math.cos(lat) * math.sin(lon)
    z = math.sin(lat)

    return x, y, z

def lat_lon_convert(x,y,z):
    '''
    Converts cartesian to latitude and longitude
    '''
    # Phi = angle formed by z and radius of sphere
    lat = math.degrees(math.atan2(z , math.sqrt(x**2 + y**2)))
    # Theta = angle formed by x and y
    lon = math.degrees(math.atan2(y,x))
        

    return lat, lon

def slerp(p1: list[float, float, float], p2: list[float, float, float], t: int) -> list[float, float, float]:
    '''
    spherical linear interpolation in cartesian coordinates.
    Input:
        [p1]    - Point 1 in cartesian coordinates.
        [p2]    - Point 2 in cartesian coordinates.
        [t]     - Value in [0,1]. 
    Returns:
        [pt]    - Point in interpolation in cartesian coordinates
    '''
    ## Unpack Cartesian ##
    x1,y1,z1 = p1 
    x2,y2,z2 = p2

    ## Dot Product ##
    dp = (x1 * x2) + (y1 * y2) + (z1 * z2)

    ## Angle between points ##
    theta = math.acos(dp)
    sin_theta = math.sin(theta)

    ## Interpolation factor ##
    interp1 = math.sin(theta * (1-t)) / sin_theta
    interp2 = math.sin(theta * (t)) / sin_theta

    ## Transform Coordinates ##
    xt = (x1 * interp1) + (x2 * interp2)
    yt = (y1 * interp1) + (y2 * interp2)
    zt = (z1 * interp1) + (z2 * interp2)

    return lat_lon_convert(xt, yt, zt)
    
def flight_trajectory(p1: [float, float], p2: [float, float], n : int) -> list[list[float, float]]: 
    '''
    Calculate flight trajectory between two locations.
    Input:
        [p1]    - Point 1. [longitude, latitude]
        [p2]    - Point 2. [longitude, latitude]
        [n]     - number of points
    Returns:
        [path]  - list of points in flight path. [longitude, latitude]
    '''
    ## Initialize Path ##
    path = []

    ## Unpack Longitude and Latitude ##
    lat1, long1 = p1 
    lat2, long2 = p2

    x1,y1,z1 = cartesian_convert(lat1, long1) 

    xn, yn, zn = cartesian_convert(lat2, long2) 

    for i in range(n+1):
        path.append(slerp([x1,y1,z1], [xn, yn, zn], i / n))

    return path



In [ ]:
# flights_flattened = df_out[["Latitude_Source", "Longitude_Source", "Latitude_Dest", "Longitude_Dest", "Source_Airport", "Destination_Airport"]].values.tolist()

# flight_paths = []

# lons = []
# lats = []

# for i in range(1600,1610):
#     [lat1, long1, lat2, long2, airport_source, airport_dest] = flights_flattened[i]
#     flight_paths = (flight_trajectory([lat1,long1], [lat2, long2], 10))
#     [lons.append(x[0]) for x in flight_paths]
#     [lats.append(x[1]) for x in flight_paths]
# # flight_test_case = [1608, "LAX", "BOS"]
lats = []
lons = []
for i in range(1000):
    [lat1, long1, lat2, long2] = df_out[['Latitude_Source', 'Longitude_Source', 'Latitude_Dest', 'Longitude_Dest']].loc[0+i]
    test_case_path = flight_trajectory([lat1,long1], [lat2, long2], 10)

    
    [lats.append(x[0]) for x in test_case_path]
    [lons.append(x[1]) for x in test_case_path] 




In [350]:
haunted_places_df = pd.read_csv("../data/processed/haunted_places_cleaned.tab", sep = "\t")
haunted_places_df.shape
haunted_places_df['Haunting_Type'] = pd.Series(np.array(['UFO'] * 5000 + ['Ghost'] * 3000 + ['Mansion'] * 2991))
haunted_places_df['Haunting_Type'].unique().tolist()

['UFO', 'Ghost', 'Mansion']

In [ ]:
import plotly.graph_objects as go

# Example coordinates (you can replace these with your own)


# Create Plotly figure
fig = go.Figure()

## Add Haunting Types ##
haunt_types = haunted_places_df['Haunting_Type'].unique().tolist()
haunting_colors = ['rgb(127,255,0)', 'rgb(218,112,214)', 'rgb(138,43,226)']
for i, haunting_type in enumerate(haunt_types):
    haunted_places_df_filtered = haunted_places_df.loc[haunted_places_df['Haunting_Type'] == haunting_type]
    fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon = haunted_places_df_filtered['longitude'],
        lat = haunted_places_df_filtered['latitude'],
        hoverinfo = 'text',
        text = haunted_places_df_filtered['Haunting_Type'],
        mode = 'markers',
        marker = dict(
            size = 2,
            color = haunting_colors[i],
            opacity = 0.5
            ),
            name = "Haunted Places",
            visible = False
        )
    )
    
## Add Flight Paths ##
fig.add_trace(go.Scattergeo(
    lon=lons,
    lat=lats,
    mode='lines',
    line=dict(width=.5, color='red'),
    opacity = 0.5, 
    hoverinfo = 'skip', 
    name = "Flights",
    visible = True
))

## Add Airports ##
fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = valid_american_airports['Longitude_Deg'],
    lat = valid_american_airports['Latitude_Deg'],
    hoverinfo = 'text',
    text = valid_american_airports['Iata_Code'],
    mode = 'markers',
    marker = dict(
        size = 2,
        color = 'rgb(0, 0, 255)',
        opacity = 0.5
        ),
        name = "Airports",
        visible = False
        ))

## Add Interactive Buttons for Haunts ##
buttons_haunts = [
            {
                "label"  : haunt_type,
                "method" : "update",
                "args"   :  [{"visible" : [haunt_type == h for h in haunt_types] + [False] + [False]}],
            }
 for haunt_type in haunt_types
]


buttons_haunts.append(
            {
                "label"  : "Show Haunts and Flight Paths",
                "method" : "update",
                "args"   :  [{"visible" : [True] * len(haunt_types) + [True, False]}],
            },
        )

buttons_haunts.append(
            {
                "label"  : "Show Haunts and Airports",
                "method" : "update",
                "args"   :  [{"visible" : [True] * len(haunt_types) + [False, True]}],
            },)

## Add Interactive Buttons for Flights ##
buttons_flights = [
            {
                "label"  : "Show Flights",
                "method" : "update",
                "args"   :  [{"visible" : [[False] * len(haunt_types) + [True, False]]}]
            },
                        {
                "label"  : "Show Airports",
                "method" : "update",
                "args"   :  [{"visible" : [[False] * len(haunt_types) + [False, True]]}]
            },
                        {
                "label"  : "Show Flights and Airports",
                "method" : "update",
                "args"   :  [{"visible" : [{"visible" : [[True] * len(haunt_types) + [True, True]]}]}]
            }]


fig.update_layout(
    title_text = 'Flight Paths Accross U.S.',
    showlegend = False,
    geo = dict(
        scope = 'north america',
        projection_type = 'azimuthal equal area',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
    ),
    updatemenus=[
    {
        "buttons": buttons_haunts,
        "direction" : "down",
        "showactive" : True,
        "x": 0.1,
        "y": 1.15,
        "xanchor" : "left",
        "yanchor" : "top", 
        "font": {"size" : 12},
        "type": "dropdown",
        "name": "Haunt Toggle"
    },
    {
        "buttons": buttons_flights,
        "direction" : "down",
        "showactive" : True,
        "x": 0.1,
        "y": 1.0,
        "xanchor" : "left",
        "yanchor" : "top", 
        "font": {"size" : 12},
        "type": "dropdown",
        "name": "Flight Toggle"
    }
    ]
)

fig.write_html("flight_paths.html")

# Show plot
fig.show()

In [368]:
import plotly.graph_objects as go
import pandas as pd

# Sample haunted places data
haunted_places_df = pd.DataFrame({
    "longitude": [-75.1652, -80.1918, -87.6298, -74.0060, -118.2437],
    "latitude": [39.9526, 25.7617, 41.8781, 40.7128, 34.0522],
    "Haunting_Type": ["Ghost", "Demonic", "Poltergeist", "Ghost", "Demonic"]
})

# Sample flight paths (lat/lon pairs)
flight_paths_df = pd.DataFrame({
    "start_lon": [-118.2437, -80.1918],
    "start_lat": [34.0522, 25.7617],
    "end_lon": [-74.0060, -87.6298],
    "end_lat": [40.7128, 41.8781]
})

# Sample airport locations
airports_df = pd.DataFrame({
    "longitude": [-77.037, -73.778, -87.904],
    "latitude": [38.852, 40.641, 41.978],
    "name": ["DCA", "JFK", "ORD"]
})

# Create figure
fig = go.Figure()

# Define category layers
categories = {
    "Haunts": [],
    "Flights": [],
    "Airports": []
}

# Add traces for each haunting type
haunting_types = haunted_places_df["Haunting_Type"].unique()
for haunt in haunting_types:
    df_filtered = haunted_places_df[haunted_places_df["Haunting_Type"] == haunt]
    
    trace = go.Scattergeo(
        locationmode='USA-states',
        lon=df_filtered['longitude'],
        lat=df_filtered['latitude'],
        hoverinfo='text',
        text=df_filtered['Haunting_Type'],
        mode='markers',
        marker=dict(size=5, opacity=0.6),
        name=f"Haunts - {haunt}",
        visible=False  # Default: hidden
    )
    
    fig.add_trace(trace)
    categories["Haunts"].append(trace)

# Add flight path traces
for i in range(len(flight_paths_df)):
    trace = go.Scattergeo(
        locationmode="USA-states",
        lon=[flight_paths_df.iloc[i]["start_lon"], flight_paths_df.iloc[i]["end_lon"]],
        lat=[flight_paths_df.iloc[i]["start_lat"], flight_paths_df.iloc[i]["end_lat"]],
        mode="lines",
        line=dict(width=2, color="red"),
        name=f"Flight Path {i+1}",
        visible=False  # Default: hidden
    )

    fig.add_trace(trace)
    categories["Flights"].append(trace)

# Add airport traces
trace_airports = go.Scattergeo(
    locationmode="USA-states",
    lon=airports_df["longitude"],
    lat=airports_df["latitude"],
    hoverinfo="text",
    text=airports_df["name"],
    mode="markers",
    marker=dict(size=6, color="purple", opacity=0.8),
    name="Airports",
    visible=False  # Default: hidden
)
fig.add_trace(trace_airports)
categories["Airports"].append(trace_airports)

# **Generate Independent Toggle Buttons**
toggle_buttons = []

for category, traces in categories.items():
    toggle_buttons.append({
        "label": f"Toggle {category}",
        "method": "update",
        "args": [
            {"visible": [
                not trace.visible  # Toggle visibility state
                if trace in traces else trace.visible  # Preserve other traces
                for trace in fig.data
            ]},
            {"title": f"Toggled: {category}"}
        ]
    })

# **Add "Show All" & "Hide All" Buttons**
toggle_buttons.append({
    "label": "Show All",
    "method": "update",
    "args": [{"visible": [True] * len(fig.data)}, {"title": "All Categories"}]
})

toggle_buttons.append({
    "label": "Hide All",
    "method": "update",
    "args": [{"visible": [False] * len(fig.data)}, {"title": "None"}]
})

# **Update Layout with Multiple Button Menus**
fig.update_layout(
    geo=dict(scope="usa", showland=True),
    updatemenus=[
        {
            "buttons": toggle_buttons,
            "direction": "down",  # Can also use "up" for dropdown-like behavior
            "showactive": True,
            "x": 0.1,
            "y": 1.15,
            "xanchor": "left",
            "yanchor": "top",
            "font": {"size": 12},
            "type": "buttons"  # Buttons allow multi-toggle
        }
    ]
)
fig.show()
# Save as HTML
# fig.write_html("multi_toggle_map.html")

,city,country,description,location,state,state_abbrev,longitude,latitude,city_longitude,city_latitude
0,Ada,United States,ada witch sometimes see misty blue figure floa...,Ada Cemetery,Michigan,MI,-85.504893,42.962106,-85.495480,42.960727
1,Addison,United States,little girl killed suddenly waiting school bus...,North Adams Rd.,Michigan,MI,-84.381843,41.971425,-84.347168,41.986434
2,Adrian,United States,take gorman rd . west towards sand creek come ...,Ghost Trestle,Michigan,MI,-84.035656,41.904538,-84.037166,41.897547
3,Adrian,United States,1970 's one room room 211 old section dorms ex...,Siena Heights University,Michigan,MI,-84.017565,41.905712,-84.037166,41.897547
4,Albion,United States,kappa delta sorority kappa delta sorority haun...,Albion College,Michigan,MI,-84.745177,42.244006,-84.753030,42.243097
...,...,...,...,...,...,...,...,...,...,...
10986,Westminster,United States,12 midnight see lady two little girls looking ...,city hall,Colorado,CO,-105.048936,39.862610,-105.037205,39.836653
10987,Westminster,United States,haunted victims murder happened years ago haun...,Pillar of Fire,Colorado,CO,-105.032091,39.847237,-105.037205,39.836653
10988,Wheat Ridge,United States,institution kids 18 years old younger . closed...,Ridge Mental Institution,Colorado,CO,-105.063974,39.769726,-105.077206,39.766098
10989,Wheat Ridge,United States,gymnasium reports little girl girls locker roo...,Wheat Ridge Middle School,Colorado,CO,-105.103613,39.764055,-105.077206,39.766098
